## Nemo Retrieval Microservice Tutorial

This notebook provides a simple end-to-end example of how to use Nemo Retreiver Microservice APIs.
# Getting Started with the Nemo Retriever "Retriever" microservice
|nickr@nvidia.com| Author(s) | [Nick Remearan](https://github.com/)
|rkharwar@nvidia.com| Author(s) | [Ruchika Kharwar](https://github.com/rasalt)

NOTE: This notebook has been tested in the following environment:
Python version = 3.10.8

## Pipelines 

A pipeline is an end-to-end retrieval function using Nvidia Retriever Microservice.
This system is accessed via a set of API calls/Client library

Here we list the pipeline names along with their status and the embedding model the pipeline is using. Notice the document store being used on the backend is part of the pipeline name. 

There are other properties of the pipelines (chunking strategy) which can also be viewed by printing out the entire pipeline object.

## Overview

<> 

## Objective
This notebook aims to show you how to leverage a freshly deployed "embedding micro-service".
These examples aim to be building blocks of the larger solution you will likley have in place for yout Generative AI use case.

## Before you begin
### Set up your environment.
Refer to page <> for details on how to deploy the service.
You should have docker services running in your environment thus  

docker                         dockerd                        dockerd-rootless-setuptool.sh  dockerd-rootless.sh            docker-proxy                   
nvidia@dev-h100-rkharwar-gpu01:~/retriever_03182024/docker-compose$ docker compose -f 
config/                      docker-compose-ea.yaml       docker-compose-nemollm.yaml  models/                      models_orig/                 volumes/                     
nvidia@dev-h100-rkharwar-gpu01:~/retriever_03182024/docker-compose$ docker compose -f docker-compose-ea.yaml ps
NAME                              IMAGE                                                                              COMMAND                  SERVICE          CREATED        STATUS                    PORTS
docker-compose-elasticsearch-1    docker.elastic.co/elasticsearch/elasticsearch:8.12.0                               "/bin/tini -- /usr/l…"   elasticsearch    21 hours ago   Up 21 hours (healthy)     0.0.0.0:9200->9200/tcp, :::9200->9200/tcp, 9300/tcp
docker-compose-embedding-ms-1     nvcr.io/ohlfw0olaadg/ea-participants/nemo-retriever-embedding-microservice:24.02   "/opt/nvidia/nvidia_…"   embedding-ms     21 hours ago   Up 21 hours (healthy)     
docker-compose-etcd-1             quay.io/coreos/etcd:v3.5.11                                                        "etcd -advertise-cli…"   etcd             21 hours ago   Up 21 hours (healthy)     2379-2380/tcp
docker-compose-milvus-1           milvusdb/milvus:v2.3.5                                                             "/tini -- milvus run…"   milvus           21 hours ago   Up 21 hours (healthy)     
docker-compose-minio-1            minio/minio:RELEASE.2023-03-20T20-16-18Z                                           "/usr/bin/docker-ent…"   minio            21 hours ago   Up 21 hours (healthy)     9000/tcp
docker-compose-otel-collector-1   otel/opentelemetry-collector-contrib:0.91.0                                        "/otelcol-contrib --…"   otel-collector   21 hours ago   Up 21 hours               0.0.0.0:4317->4317/tcp, :::4317->4317/tcp, 0.0.0.0:13133->13133/tcp, :::13133->13133/tcp, 0.0.0.0:55679->55679/tcp, :::55679->55679/tcp, 55678/tcp
docker-compose-postgres-1         postgres:16.1                                                                      "docker-entrypoint.s…"   postgres         21 hours ago   Up 21 hours               0.0.0.0:5432->5432/tcp, :::5432->5432/tcp
docker-compose-retrieval-ms-1     nvcr.io/ohlfw0olaadg/ea-participants/nemo-retriever-microservice:24.02             "/usr/bin/shelless_u…"   retrieval-ms     21 hours ago   Up 21 hours (unhealthy)   0.0.0.0:1984->8000/tcp, :::1984->8000/tcp
docker-compose-tika-1             apache/tika:2.9.1.0                                                                "/bin/sh -c 'exec ja…"   tika             21 hours ago   Up 21 hours               0.0.0.0:9998->9998/tcp, :::9998->9998/tcp
docker-compose-zipkin-1           openzipkin/zipkin:3.0.6                                                            "start-zipkin"           zipkin           21 hours ago   Up 21 hours (healthy)     9410/tcp, 0.0.0.0:9411->9411/tcp, :::9411->9411/tcp

### Setup the environment vairables

In [ ]:
PIPELINE_URL = "http://localhost:1984/v1"

### Package imports

In [ ]:
import sys
sys.path.insert(0, '../utils/')

from request_utils import *

## Pipelines

NeMo Retriever comes preloaded with several pipelines. Pipelines fully encapsulate the indexing and query logic from chunking to embedding model specifics to which vector store and index to use, and more.

In this section we see the types and configuration settings of the pipelines.

In [ ]:
response = get_api(PIPELINE_URL + "/pipelines")

for pipeline in response['pipelines']:
    print(pipeline)
    print(pipeline['id'])
    print("\tenabled=" + str(pipeline['enabled']))
    print("\tmodel=" + str(pipeline['config']['index']['pipeline']['components']['embedder']['init_parameters']))
    print("\tmodel=" + str(pipeline['config']['index']['pipeline']['components']['splitter']))
    print("-" * 80)    

## Collections

A collection refers to a set of uploaded documents. Using collections allows us to query against different subsets of documents.

Before we create a collection, let's specify a few items we will need:
1. Collection name
2. Pipeline type for the collection (we saw our options in the previous step)
3. Document to add to the collection
4. Query we want to do retrieval on
5. Number of chunks we want retrieved for our query (top k)

In [ ]:
### Download a document of interest.

In [ ]:
! wget -O "CUDA.pdf" -nc --user-agent="Mozilla" https://docs.nvidia.com/cuda/pdf/CUDA_C_Programming_Guide.pdf

In [ ]:
### Setup the environment vairables

In [ ]:
COLLECTION_NAME = "cuda"
PIPELINE_TYPE = "hybrid"

### Here we create our collection using the pipeline type we specified above.

In [ ]:
collection = {
    "name": COLLECTION_NAME,
    "pipeline": PIPELINE_TYPE
}

response = post_api(PIPELINE_URL+"/collections", collection)
print(response)

Let's double check to make sure our collection was created.

In [ ]:
id = response['collection']['id']
response = get_api(PIPELINE_URL + "/collections/" + id)
print(response)

### Add a document to the collection

In [ ]:
document = "CUDA.pdf"
response = upload_doc(PIPELINE_URL, id, [f"name={document}"], document)
print(response)

Let's query our retrieval pipeline now on the document we just added and request topk chunks to be returned to us.

In [ ]:
query = "what is the __global__ execution space specifier?"
topk = 5

retrieve = {
    "query": query,
    "top_k": topk
}
response = post_api(PIPELINE_URL + "/collections/" + id + "/search", retrieve)
print(response)

In [ ]:
large_chunk = ""
for chunk in response['chunks']:
    print(f"chunk len (chars): {len(chunk['content'])}")
    large_chunk += f" {chunk}"

print(f"context len (chars): {len(large_chunk)}")

In [ ]:
#invoke_url = "https://api.nvcf.nvidia.com/v2/nvcf/pexec/functions/8f4118ba-60a8-4e6b-8574-e38a4067a4a3" #Mixtral 8x7B
NVCF_BASE_URL = "https://integrate.api.nvidia.com/v1"
NGC_API_KEY = ""

from openai import OpenAI
client = OpenAI(
  base_url = NVCF_BASE_URL,
  api_key = NGC_API_KEY
)

In [ ]:
PROMPT_TEMPLATE = (
 "<s>[INST] <<SYS>>"
 "{system_prompt}"
 "<</SYS>>"
 ""
 "[Question]"
 "{question}"
 "[The Start of the Reference Context]"
 "{ctx_ref}"
 "[The End of Reference Context][/INST]"
)

system_prompt = """
You are a helpful AI assistant.
You will reply to questions only based on the reference context that you are provided.
If you cannot answer the question using only the reference context then you will politely respond that you are unable to answer the question given the provided information.
"""

In [ ]:

import json
try:
    prompt = PROMPT_TEMPLATE.format(system_prompt=system_prompt, question=query, ctx_ref=large_chunk)
    payload = {
        "messages": [
            {
            "content": prompt,
            "role": "user"
            }
        ],
        "temperature": 0.2,
        "top_p": 0.7,
        "max_tokens": 500,
        "stream": True
        }
    completion = client.chat.completions.create(
      model="mistralai/mixtral-8x7b-instruct-v0.1",
      messages=[{"role":"user","content":prompt}],
      temperature=0.5,
      top_p=1,
      max_tokens=1024,
      stream=True
    )
    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")


except Exception as e:
    print("Exception:", e)

### Finally, let's clean up by removing any collections.

In [ ]:
for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

    for line in response.iter_lines():
        if line:
            print(line)
            json_part = line.decode('utf-8').split("data: ", 1)[1]
            data = json.loads(json_part)
            print(data)
            print(data["choices"][0]["delta"]["content"], end='', flush=True)


In [29]:
response = get_api(PIPELINE_URL + "/collections")
print(response)

for collection in response['collections']:
    delete_api(PIPELINE_URL + "/collections/"+collection['id'])

response = get_api(PIPELINE_URL + "/collections")
print(response)

{'collections': [{'pipeline': 'hybrid', 'name': 'testCollection', 'id': 'c869ce4e-f73a-4f56-bc70-c9effb59f9e6'}, {'pipeline': 'hybrid', 'name': 'testCollection', 'id': '24ad6245-6ad0-46f4-8d3b-d935df57480d'}, {'pipeline': 'hybrid', 'name': 'testCollection', 'id': '85238fa8-e78f-4d64-b6b2-0f5efe1e2068'}, {'pipeline': 'hybrid', 'name': 'testCollection', 'id': 'daf90a93-6a1c-407d-9241-d4a5cac11e7f'}]}
{'collections': []}
